# **EJERCICIO 1**

In [ ]:
# ------------------------------------------------------------------------------
# 🧭 EDA de S&P 500 (Curso de Minería de Datos)
# ------------------------------------------------------------------------------

# Este cuaderno realiza un **Análisis Exploratorio de Datos (EDA)** sobre:
# - sp500_data.csv
# - sp500_sectors.csv
#
# Objetivos:
# 1) Revisar tipos de variables
# 2) Detectar datos nulos
# 3) Análisis descriptivo (tendencia central y dispersión)
# 4) Distribuciones e identificación de outliers
# 5) Tablas de frecuencias
# 6) Correlación binaria (Pearson y Spearman)
# ------------------------------------------------------------------------------


# %%
# 🔧 Instalación y carga de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)

print("Librerías cargadas.")


In [ ]:
# %%
# 📥 Lectura de archivos
DATA_PATH = Path('.')  # en Colab: '.' es el directorio de trabajo
file_main    = DATA_PATH / 'sp500_data.csv'
file_sectors = DATA_PATH / 'sp500_sectors.csv'

df = pd.read_csv(file_main)
df_sectors = pd.read_csv(file_sectors)

print('Tamaños:')
print('sp500_data:', df.shape)
print('sp500_sectors:', df_sectors.shape)

print("\nPrimeras filas de sp500_data:")
display(df.head())



In [ ]:
print("\nPrimeras filas de sp500_sectors:")
display(df_sectors.head())

In [ ]:
print("\nUltimas filas de sp500_sectors:")
display(df_sectors.tail())

In [ ]:
# %%
# 🔎 Inspección inicial: esquema y tipos de variables
print('Tipos de datos (sp500_data):')
print(df.dtypes)

print("\nConteo de nulos (sp500_data):")
print(df.isna().sum())

print("\nTipos de datos (sp500_sectors):")
print(df_sectors.dtypes)

print("\nConteo de nulos (sp500_sectors):")
print(df_sectors.isna().sum())

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print("\nColumnas numéricas:", num_cols)
print("Columnas no numéricas:", cat_cols)

In [ ]:
# Muestra de valores únicos en categóricas
max_show = 15
for c in cat_cols:
    uniques = df[c].dropna().unique()
    print(f"\nColumna '{c}': {len(uniques)} valores únicos (muestra hasta {max_show})")
    print(uniques[:max_show])

In [ ]:
# %%
# 🧩 Enriquecimiento: unión con metadatos de sector (opcional)
candidate_keys = ['Ticker', 'Symbol', 'ticker', 'symbol']
key = None
for k in candidate_keys:
    if k in df.columns and k in df_sectors.columns:
        key = k
        break

if key is not None:
    print(f'Uniendo por clave: {key!r}')
    df_eda = df.merge(df_sectors, on=key, how='left', suffixes=('', '_sector'))
else:
    print('No se detectó clave común. Continuamos solo con sp500_data.')
    df_eda = df.copy()

print("Tamaño del dataset para EDA:", df_eda.shape)
display(df_eda.head())

In [ ]:
# %%
# 📊 Análisis descriptivo: tendencia central y dispersión


extra_stats = pd.DataFrame({
    # Media aritmética: promedio de los valores de cada variable numérica
    'mean': df_eda[num_cols].mean(),

    # Mediana: valor central de la distribución (robusto frente a outliers)
    'median': df_eda[num_cols].median(),

    # Desviación estándar: medida de dispersión respecto a la media
    'std': df_eda[num_cols].std(),

    # Varianza: cuadrado de la desviación estándar (otra medida de dispersión)
    'var': df_eda[num_cols].var(),

    # Primer cuartil (Q1): valor por debajo del cual se encuentra el 25% de los datos
    'q1': df_eda[num_cols].quantile(0.25),

    # Tercer cuartil (Q3): valor por debajo del cual se encuentra el 75% de los datos
    'q3': df_eda[num_cols].quantile(0.75),

    # Asimetría (skewness): mide si la distribución es simétrica o está sesgada a un lado
    'skew': df_eda[num_cols].skew(),

    # Curtosis (kurtosis): mide el “peso” de las colas respecto a una normal (valores extremos)
    'kurt': df_eda[num_cols].kurt()
})

display(extra_stats)

In [ ]:
desc = df_eda.describe(include='all').T
display(desc)

In [ ]:
# %%
# 📋 Tablas de frecuencias para categóricas
cat_cols_eda = df_eda.select_dtypes(exclude=[np.number]).columns.tolist()
for c in cat_cols_eda:
    print(f"\n=== {c} ===")
    vc = df_eda[c].value_counts(dropna=False)
    vcp = df_eda[c].value_counts(normalize=True, dropna=False).round(4)
    freq = pd.DataFrame({'freq_abs': vc, 'freq_rel': vcp})
    display(freq.head(30))


In [ ]:
# %%
# 🔔 Distribución de datos: histogramas y boxplots
#
# OJO: sólo estamos viendo las tres primeras columnas numéricas
#
for c in num_cols[:3]:
    # Histograma
    plt.figure()
    df_eda[c].dropna().hist(bins=30)
    plt.title(f'Histograma de {c}')
    plt.xlabel(c)
    plt.ylabel('Frecuencia')
    plt.show()

    # Boxplot
    plt.figure()
    plt.boxplot(df_eda[c].dropna(), vert=True)
    plt.title(f'Boxplot de {c}')
    plt.ylabel(c)
    plt.show()

In [ ]:
# %%
# 🚨 Detección de outliers (IQR)
iqr_report = []
outlier_indices = {}
#
# OJO: sólo estamos viendo las cinco primeras columnas numéricas
#
for c in num_cols[:5]:
    s = df_eda[c].dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    low  = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    mask = (df_eda[c] < low) | (df_eda[c] > high)
    count = int(mask.sum())
    iqr_report.append({'variable': c, 'q1': q1, 'q3': q3, 'iqr': iqr,
                       'low': low, 'high': high, 'n_outliers': count})
    outlier_indices[c] = df_eda.index[mask].tolist()

iqr_df = pd.DataFrame(iqr_report).sort_values('n_outliers', ascending=False)
display(iqr_df)

# outlier_indices['NombreDeVariable']  # para ver índices específicos

In [ ]:
# %%
# 🕳️ Patrón de valores nulos
nulls = df_eda.isna().sum().sort_values(ascending=False)
pct_nulls = (df_eda.isna().mean()*100).round(2).sort_values(ascending=False)

nulls_df = pd.DataFrame({'n_nulls': nulls, 'pct_nulls': pct_nulls})
display(nulls_df)

print("\nNúmero de filas con algún nulo:", int(df_eda.isna().any(axis=1).sum()))
rows_with_nulls = df_eda[df_eda.isna().any(axis=1)].head(10)
display(rows_with_nulls)

In [ ]:
# %%
# 🔗 Correlación binaria (Pearson y Spearman)
#
# OJO: sólo estamos viendo las cinco primeras columnas numéricas
#
num_df = df_eda[num_cols[:15]].select_dtypes(include=[np.number]).copy()

corr_pearson = num_df.corr(method='pearson')
corr_spearman = num_df.corr(method='spearman')

print("Correlación (Pearson):")
display(corr_pearson)

print("\nCorrelación (Spearman):")
display(corr_spearman)

In [ ]:
# Heatmap Pearson
plt.figure()
plt.imshow(corr_pearson, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Mapa de calor - Pearson")

# Ejes
plt.xticks(range(len(corr_pearson.columns)), corr_pearson.columns, rotation=90)
plt.yticks(range(len(corr_pearson.index)), corr_pearson.index)
plt.colorbar()

# Anotar los valores en cada celda
for i in range(len(corr_pearson.index)):
    for j in range(len(corr_pearson.columns)):
        val = corr_pearson.iloc[i, j]
        plt.text(j, i, f"{val:.2f}", ha="center", va="center", color="black")

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap Spearman
plt.figure()
plt.imshow(corr_spearman, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Mapa de calor - Spearman")

# Ejes
plt.xticks(range(len(corr_spearman.columns)), corr_spearman.columns, rotation=90)
plt.yticks(range(len(corr_spearman.index)), corr_spearman.index)
plt.colorbar()

# Anotar los valores en cada celda
for i in range(len(corr_spearman.index)):
    for j in range(len(corr_spearman.columns)):
        val = corr_spearman.iloc[i, j]
        plt.text(j, i, f"{val:.2f}", ha="center", va="center", color="black")

plt.tight_layout()
plt.show()


In [ ]:
# %%
# 🧪 Comparaciones bivariadas rápidas (opcional)
x_var = 'INTC'  # cambia aquí
y_var = 'ALTR'  # cambia aquí

if x_var in num_cols and y_var in num_cols and x_var != y_var:
    plt.figure()
    plt.scatter(df_eda[x_var], df_eda[y_var])
    plt.xlabel(x_var); plt.ylabel(y_var)
    plt.title(f'Dispersión: {x_var} vs {y_var}')
    plt.show()
else:
    print("Define x_var y y_var con nombres de columnas numéricas distintas.")

# **EJERCICIO 2**

In [ ]:
# %%
# 📥 Lectura de archivos
DATA_PATH = Path('.')  # en Colab: '.' es el directorio de trabajo
file_main    = DATA_PATH / 'state.csv'


df_state = pd.read_csv(file_main)


print('Tamaños:')
print('state:', df_state.shape)


print("\nPrimeras filas de sp500_data:")
display(df_state.head())


In [ ]:
# %%
# 📊 Estadísticos descriptivos
desc = df_state.describe(include='all').T
display(desc)



In [ ]:
# Variabilidad y percentiles de Population y Murder.Rate
variability = pd.DataFrame({
    'mean': df_state[['Population','Murder.Rate']].mean(),
    'std': df_state[['Population','Murder.Rate']].std(),
    'var': df_state[['Population','Murder.Rate']].var(),
    'q1': df_state[['Population','Murder.Rate']].quantile(0.25),
    'median': df_state[['Population','Murder.Rate']].median(),
    'q3': df_state[['Population','Murder.Rate']].quantile(0.75),
    'min': df_state[['Population','Murder.Rate']].min(),
    'max': df_state[['Population','Murder.Rate']].max(),
    'skew': df_state[['Population','Murder.Rate']].skew(),
    'kurt': df_state[['Population','Murder.Rate']].kurt()
})
display(variability)

In [ ]:
# %%
# 🚨 Boxplots para ver dispersión y outliers
for col in ['Population','Murder.Rate']:
    plt.figure()
    plt.boxplot(df_state[col], vert=True)
    plt.title(f'Boxplot de {col}')
    plt.ylabel(col)
    plt.show()


In [ ]:
# %%
# 📋 Tablas de frecuencias por rangos de población
# Definimos intervalos (bins) de población
bins = [0, 2_000_000, 5_000_000, 10_000_000, 20_000_000, 40_000_000]
labels = ['0-2M','2-5M','5-10M','10-20M','20-40M']

df_state['PopRange'] = pd.cut(df_state['Population'], bins=bins, labels=labels, include_lowest=True)

freq_abs = df_state['PopRange'].value_counts().sort_index()
freq_rel = df_state['PopRange'].value_counts(normalize=True).sort_index()

freq_table = pd.DataFrame({'Frecuencia Absoluta': freq_abs, 'Frecuencia Relativa': freq_rel.round(3)})
display(freq_table)


In [ ]:
# %%
# 📉 Histogramas con curva normal teórica

from scipy.stats import norm
import numpy as np

# --- Population ---
plt.figure()
# Histograma con densidad
df_state['Population'].hist(bins=10, density=True, alpha=0.6)

# Parámetros de la normal
mu, sigma = df_state['Population'].mean(), df_state['Population'].std()

# Rango de valores
x = np.linspace(df_state['Population'].min(), df_state['Population'].max(), 200)
plt.plot(x, norm.pdf(x, mu, sigma), 'r-', lw=2, label='Curva normal')

plt.title('Histograma de Population con curva normal')
plt.xlabel('Population')
plt.ylabel('Densidad')
plt.legend()
plt.show()


# --- Murder.Rate ---
plt.figure()
df_state['Murder.Rate'].hist(bins=10, density=True, alpha=0.6)

mu, sigma = df_state['Murder.Rate'].mean(), df_state['Murder.Rate'].std()
x = np.linspace(df_state['Murder.Rate'].min(), df_state['Murder.Rate'].max(), 200)
plt.plot(x, norm.pdf(x, mu, sigma), 'r-', lw=2, label='Curva normal')

plt.title('Histograma de Murder.Rate con curva normal')
plt.xlabel('Murder.Rate')
plt.ylabel('Densidad')
plt.legend()
plt.show()


In [ ]:
# %%
# 🔥 Mapa de calor: Estados vs (Population, Murder.Rate) ordenado por tasa de asesinatos
# Ordenamos de menor a mayor tasa de asesinatos
df_sorted = df_state.sort_values('Murder.Rate', ascending=False).reset_index(drop=True)

# Seleccionamos columnas numéricas a mostrar
heat_data = df_sorted[['Population', 'Murder.Rate']]

plt.figure(figsize=(8, 12))
plt.imshow(heat_data, aspect='auto', cmap='YlOrRd')
plt.colorbar(label='Valor')

# Etiquetas de ejes
plt.xticks(range(heat_data.shape[1]), heat_data.columns, rotation=45)
plt.yticks(range(len(df_sorted['State'])), df_sorted['State'])

# Anotar los valores dentro de cada celda
for i in range(heat_data.shape[0]):  # filas (estados)
    for j in range(heat_data.shape[1]):  # columnas (variables)
        val = heat_data.iloc[i, j]
        plt.text(j, i,
                 f"{val:,.0f}" if j == 0 else f"{val:.1f}",  # formato distinto para población y asesinatos
                 ha="center", va="center", color="black", fontsize=8)

plt.title("Mapa de calor: Estados ordenados por tasa de asesinatos")
plt.tight_layout()
plt.show()




In [ ]:
# %%
# 🎯 Gráfico de dispersión
plt.figure()
plt.scatter(df_state['Population'], df_state['Murder.Rate'])
plt.xlabel("Population")
plt.ylabel("Murder.Rate")
plt.title("Dispersión: Population vs Murder.Rate")
plt.show()


In [ ]:
# %%
# 🎯 Gráfico de dispersión con etiquetas de los estados
plt.figure(figsize=(10,6))
plt.scatter(df_state['Population'], df_state['Murder.Rate'])

# Etiquetas de los estados
for i, row in df_state.iterrows():
    plt.text(row['Population'], row['Murder.Rate'], row['State'], fontsize=8)

plt.xlabel("Population")
plt.ylabel("Murder.Rate")
plt.title("Dispersión: Population vs Murder.Rate (con etiquetas)")
plt.show()
